In [198]:
from imblearn.over_sampling import SMOTE #This is for SMOTE
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,roc_curve,roc_auc_score
# from catboost import CatBoostClassifier
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
# from xgboost import XGBClassifier
import pickle
from sklearn.preprocessing import LabelEncoder

In [199]:
df = pd.read_csv('final_fullset_cleared.csv')

In [200]:
df

,year,city,season,area(hec),production(MT),tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,windspeed,winddir,sealevelpressure,cloudcover,visibility
0,2009,Kurunegala,1,491.00,1695,31.426257,21.556983,27.377654,35.978212,21.556983,29.975978,22.402235,75.672737,20.652514,169.201117,1009.093296,45.236872,11.863128
1,2009,Kurunegala,2,384.00,3648,30.735119,21.498810,27.035714,35.520833,21.498810,29.496429,23.038690,79.574762,19.487500,187.786905,1009.424405,57.911310,12.922024
2,2010,Kurunegala,1,504.00,1715,32.743094,21.376796,27.580110,37.991160,21.376796,30.455249,22.895580,77.036298,20.113260,164.210497,1009.548619,50.776796,13.509392
3,2010,Kurunegala,2,414.00,3929,30.269022,21.120109,26.505435,34.859783,21.120109,28.627717,22.838587,80.983152,19.066304,202.689674,1008.176087,67.685870,13.566304
4,2011,Kurunegala,1,496.00,1587,31.633523,21.084659,27.017045,36.293750,21.084659,29.450568,22.651136,78.151420,20.335795,165.268750,1008.642045,60.318750,13.209659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2016,Nuwaraeliya,1,59.00,1262,19.002312,13.518497,15.824855,19.028902,13.518497,15.869942,13.206936,85.348092,11.668786,233.346821,1010.005780,65.633526,17.602312
116,2017,Nuwaraeliya,1,27.61,394,20.706897,12.003448,15.935632,20.785632,12.000000,15.989655,11.847126,79.532701,9.758621,191.841379,1010.048851,55.951724,16.705747
117,2017,Nuwaraeliya,2,31.00,451,19.253333,13.970000,16.192778,19.252778,13.970000,16.232778,14.115556,88.057167,10.872222,232.321111,1009.157222,75.410556,16.677778
118,2018,Nuwaraeliya,1,9.00,235,20.076437,12.905747,16.143103,20.089080,12.905747,16.186207,13.051149,83.634310,12.139655,177.602874,1009.322414,61.379310,18.171264


In [201]:
corr = df.corr()
corr.style.background_gradient(cmap ='coolwarm')

,year,season,area(hec),production(MT),tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,windspeed,winddir,sealevelpressure,cloudcover,visibility
year,1.000000,-0.072786,-0.186439,-0.071596,0.016304,-0.006929,0.005927,0.039907,-0.008004,0.003529,-0.000695,-0.045692,-0.103561,-0.047461,0.477478,-0.163047,0.003318
season,-0.072786,1.000000,0.176128,0.339033,-0.159779,0.035609,-0.056854,-0.117324,0.033864,-0.064999,0.048945,0.520993,0.024859,0.606298,-0.256986,0.769187,-0.086243
area(hec),-0.186439,0.176128,1.000000,0.752533,0.312890,0.053625,0.258021,0.286889,0.027120,0.246640,0.250245,-0.180513,0.228457,-0.028728,-0.275195,0.027880,-0.311784
production(MT),-0.071596,0.339033,0.752533,1.000000,0.172952,0.037838,0.154515,0.143886,0.016460,0.123659,0.179107,0.029574,-0.011609,0.212548,-0.127759,0.201313,-0.112736
tempmax,0.016304,-0.159779,0.312890,0.172952,1.000000,0.881617,0.967746,0.974022,0.870712,0.954345,0.937462,-0.709532,0.317912,-0.389034,-0.052469,-0.511546,-0.454806
tempmin,-0.006929,0.035609,0.053625,0.037838,0.881617,1.000000,0.955574,0.915804,0.998708,0.945720,0.969062,-0.492064,0.281337,-0.165467,-0.067694,-0.284540,-0.394698
temp,0.005927,-0.056854,0.258021,0.154515,0.967746,0.955574,1.000000,0.988041,0.948535,0.991899,0.986817,-0.643159,0.360040,-0.318260,-0.087582,-0.416658,-0.493621
feelslikemax,0.039907,-0.117324,0.286889,0.143886,0.974022,0.915804,0.988041,1.000000,0.909223,0.990603,0.962617,-0.701148,0.428716,-0.411469,-0.065216,-0.495332,-0.551391
feelslikemin,-0.008004,0.033864,0.027120,0.016460,0.870712,0.998708,0.948535,0.909223,1.000000,0.942868,0.960421,-0.497716,0.303246,-0.176302,-0.065588,-0.285780,-0.412858
feelslike,0.003529,-0.064999,0.246640,0.123659,0.954345,0.945720,0.991899,0.990603,0.942868,1.000000,0.972890,-0.672823,0.465084,-0.371249,-0.099534,-0.438847,-0.580884


In [202]:
df.columns

Index(['year', 'city', 'season', 'area(hec)', 'production(MT)', 'tempmax',
       'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew',
       'humidity', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility'],
      dtype='object')

In [203]:
label_encoder = LabelEncoder()

def label_encoder_converter(df_in):
    
    df['city'] = label_encoder.fit_transform(df['city'])


label_encoder_converter(df)

In [204]:
# Splitting the dataset in to x and y
x=df[['season', 'area(hec)', 'tempmax',
       'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew',
       'humidity', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility']]
y=df['production(MT)']

In [205]:
x

,season,area(hec),tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,windspeed,winddir,sealevelpressure,cloudcover,visibility
0,1,491.00,31.426257,21.556983,27.377654,35.978212,21.556983,29.975978,22.402235,75.672737,20.652514,169.201117,1009.093296,45.236872,11.863128
1,2,384.00,30.735119,21.498810,27.035714,35.520833,21.498810,29.496429,23.038690,79.574762,19.487500,187.786905,1009.424405,57.911310,12.922024
2,1,504.00,32.743094,21.376796,27.580110,37.991160,21.376796,30.455249,22.895580,77.036298,20.113260,164.210497,1009.548619,50.776796,13.509392
3,2,414.00,30.269022,21.120109,26.505435,34.859783,21.120109,28.627717,22.838587,80.983152,19.066304,202.689674,1008.176087,67.685870,13.566304
4,1,496.00,31.633523,21.084659,27.017045,36.293750,21.084659,29.450568,22.651136,78.151420,20.335795,165.268750,1008.642045,60.318750,13.209659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,59.00,19.002312,13.518497,15.824855,19.028902,13.518497,15.869942,13.206936,85.348092,11.668786,233.346821,1010.005780,65.633526,17.602312
116,1,27.61,20.706897,12.003448,15.935632,20.785632,12.000000,15.989655,11.847126,79.532701,9.758621,191.841379,1010.048851,55.951724,16.705747
117,2,31.00,19.253333,13.970000,16.192778,19.252778,13.970000,16.232778,14.115556,88.057167,10.872222,232.321111,1009.157222,75.410556,16.677778
118,1,9.00,20.076437,12.905747,16.143103,20.089080,12.905747,16.186207,13.051149,83.634310,12.139655,177.602874,1009.322414,61.379310,18.171264


In [206]:
y

0      1695
1      3648
2      1715
3      3929
4      1587
       ... 
115    1262
116     394
117     451
118     235
119     344
Name: production(MT), Length: 120, dtype: int64

In [207]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
# regressor = RandomForestRegressor(n_estimator = 10, random_state = 0)
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(x, y) 

RandomForestRegressor(random_state=0)

In [208]:
parameters = {'depth'         : [4,5,6,7,8,9, 10],'learning_rate' : [0.01,0.02,0.03,0.04],'iterations'    : [10, 20,30,40,50,60,70,80,90, 100] }
Grid_CBC = GridSearchCV(estimator=regressor, param_grid = parameters, cv = 5, n_jobs=-1)

In [209]:
# fit the regressor with x and y data
regressor.fit(x, y) # Fitting Random Forest Regression to the dataset
# import the regressor


RandomForestRegressor(random_state=0)

In [210]:
y_pred=regressor.predict(x_test)

In [211]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)

51849.946979166685

In [212]:
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10, None], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

In [213]:
rfr = RandomForestRegressor(random_state = 1)

In [214]:
g_search = GridSearchCV(estimator = rfr, param_grid = param_grid, 

                          cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

In [215]:
g_search.fit(x_train, y_train);
print(g_search.best_params_)

{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 10}


In [216]:
print(g_search.score(x_test, y_test))

0.5190131278312451
